In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
org_df = pd.read_csv('glassdoor_jobs.csv', index_col= 0 , na_values= '-1')
df = org_df.copy()
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973.0,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),NaN
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984.0,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),NaN
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010.0,Company - Private,Security Services,Business Services,$100 to $500 million (USD),NaN
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965.0,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [3]:
df.shape

(956, 14)

In [4]:
df.isnull().sum()

Job Title              0
Salary Estimate      214
Job Description        0
Rating                34
Company Name           0
Location               0
Headquarters          11
Size                  11
Founded               97
Type of ownership     11
Industry              39
Sector                39
Revenue               11
Competitors          634
dtype: int64

### Data cleaning

#### Clean salary values

In [5]:
df.dropna(inplace=True , subset= ['Salary Estimate']) # drop rows with -1 as salary value

In [6]:
# make new column for some hourly salary and employer_provided salary
df['hourly salary'] = df['Salary Estimate'].apply(lambda x : 1 if 'per hour' in x.lower() else 0)
df['employer_provided'] = df['Salary Estimate'].apply(lambda x : 1 if 'employer provided salary:' in x.lower() else 0)

In [7]:
# remove extra word from salary

salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
salary = salary.apply(lambda x: x.replace('K','').replace('$',''))
salary = salary.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))


In [8]:
# add column for min , max salary and average salary
df['min_salary'] = salary.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = salary.apply(lambda x: int(x.split('-')[1]))
df['avg_salary'] = (df.min_salary + df.max_salary)/2


#### Clean Company values

In [9]:
df['Company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-4], axis =1)

#### Clean Location values

In [10]:
df['job_state'] = df['Location'].apply(lambda x : x.split(',')[1])

In [11]:
df.job_state.value_counts()

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 NC              21
 WA              21
 NJ              17
 FL              16
 OH              14
 TN              13
 DC              11
 CO              11
 WI              10
 IN              10
 UT              10
 AZ               9
 MO               9
 AL               8
 KY               6
 MI               6
 DE               6
 GA               6
 IA               5
 CT               5
 LA               4
 OR               4
 NE               4
 KS               3
 NM               3
 MN               2
 ID               2
 SC               1
 Los Angeles      1
 RI               1
Name: job_state, dtype: int64

In [12]:
# check if a company has Headquarter in same state as given in company name or differen 
df['is_HQ_state_same'] = df.apply(lambda x : 1 if x.Location == x.Headquarters else 0,  axis =1)

#### Clean Age of company 

In [13]:
df['age_of_company'] = df.Founded.apply(lambda x: x if x < 1 else 2020 -x)

In [14]:
df.reset_index(inplace= True)

In [15]:
df.drop(['index'], axis = 1, inplace=True)

#### Title simplification

In [16]:
def title_simp(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'

def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr.' in title.lower() or 'lead' in title.lower() or 'prinicipal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'jr'
    else:
        return 'na'


In [17]:
df['job_simp'] = df['Job Title'].apply(title_simp)

df.job_simp.value_counts()

data scientist    279
na                184
data engineer     119
analyst           102
mle                22
manager            22
director           14
Name: job_simp, dtype: int64

In [18]:
df['seniority'] = df['Job Title'].apply(seniority)

df.seniority.value_counts()

na        544
senior    195
jr          3
Name: seniority, dtype: int64

In [19]:
# fix stae los Angles
df.job_state.value_counts()

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 NC              21
 WA              21
 NJ              17
 FL              16
 OH              14
 TN              13
 DC              11
 CO              11
 WI              10
 IN              10
 UT              10
 AZ               9
 MO               9
 AL               8
 KY               6
 MI               6
 DE               6
 GA               6
 IA               5
 CT               5
 LA               4
 OR               4
 NE               4
 KS               3
 NM               3
 MN               2
 ID               2
 SC               1
 Los Angeles      1
 RI               1
Name: job_state, dtype: int64

In [20]:
df['job_state'] = df.job_state.apply(lambda x: x.strip() if x.strip().lower() != 'los angeles' else 'LA')

df.job_state.value_counts()

CA    151
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
WA     21
NC     21
NJ     17
FL     16
OH     14
TN     13
DC     11
CO     11
WI     10
UT     10
IN     10
AZ      9
MO      9
AL      8
GA      6
KY      6
DE      6
MI      6
CT      5
IA      5
LA      5
OR      4
NE      4
KS      3
NM      3
ID      2
MN      2
SC      1
RI      1
Name: job_state, dtype: int64

In [21]:
# Job Description length

df['desc_length'] =  df['Job Description'].apply(lambda x : len(x)) 
df['desc_length']

0      2536
1      4783
2      3461
3      3883
4      2728
       ... 
737    6162
738    6130
739    3078
740    1642
741    3673
Name: desc_length, Length: 742, dtype: int64

In [22]:
# Competitor count

df['Competitors'] = df['Competitors'].replace(np.nan, '0')
df['num_comp'] = df['Competitors'].apply(lambda x : len(x.split(',')) if x!= '0' else 0 )

df.num_comp.value_counts()

0    460
3    228
2     41
1     12
4      1
Name: num_comp, dtype: int64

In [23]:
# Hourly wage to annual wage
df['min_salary'] = df.apply(lambda x: x.min_salary*2 if x['hourly salary'] == 1 else x.min_salary, axis=1 )

In [24]:
df['max_salary'] = df.apply(lambda x: x.max_salary*2 if x['hourly salary'] == 1 else x.max_salary, axis=1 )

In [25]:
df[df['hourly salary'] == 1][['hourly salary','min_salary', 'max_salary']]

,hourly salary,min_salary,max_salary
165,1,34,48
177,1,42,68
197,1,36,50
202,1,42,68
210,1,30,50
246,1,34,48
271,1,42,68
337,1,36,50
345,1,48,78
357,1,42,68


In [27]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,max_salary,avg_salary,Company_txt,job_state,is_HQ_state_same,age_of_company,job_simp,seniority,desc_length,num_comp
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973.0,Company - Private,...,91,72.0,Tecolote Research,NM,0,47.0,data scientist,na,2536,0
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984.0,Other Organization,...,112,87.5,University of Maryland Medical System,MD,0,36.0,data scientist,na,4783,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010.0,Company - Private,...,90,85.0,KnowBe4,FL,1,10.0,data scientist,na,3461,0
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965.0,Government,...,97,76.5,PNNL,WA,1,55.0,data scientist,na,3883,3
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998.0,Company - Private,...,143,114.5,Affinity Solutions,NY,1,22.0,data scientist,na,2728,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830.0,Company - Public,...,111,84.5,GSK,MA,0,190.0,na,senior,6162,3
738,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006.0,Company - Public,...,133,102.5,Eventbrite,TN,0,14.0,data engineer,senior,6130,3
739,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984.0,College / University,...,91,73.5,Software Engineering Institute,PA,1,36.0,na,na,3078,0
740,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,NaN,Company - Private,...,160,127.5,"Numeric, LLC",PA,0,NaN,manager,na,1642,0


In [26]:
df.to_csv('cleaned_DS_salary.csv')